# Database 5

## M:N (User-User)  


<b>개요</b>

User 자기 자신과의 M:N 관계 설정을 통한 팔로우 기능 구현하기

ex) follower

### Profile

<b>개요</b>  

자연스러운 follow 흐름을 위한 프로필 페이지를 먼저 작성

<b>Profile 구현</b>  

url 및 view 함수 작성  

```py
# accounts/urls.py

urlpatterns = [
    ...
    path('<int:article_pk>/likes/', views.likes, name='likes'),
]

```

```py
# accounts/views.py

from django.contrib.auth import get_user_model

def profile(request, username):
    User = get_user_model()
    person = User.objects.get(username=username)
    context = {'person': person,}
    
    return render(request, 'accounts/profile.html', context)
```

profile 템플릿 작성  

```html
<!-- accounts/templates/accounts/profile.html -->

{% extends 'base.html' %}

{% block content %}
  <h1>{{ person.username }} 님의 프로필</h1>

  <hr>

  <h2>{{ person.username }}'s 게시글</h2>
  {% for article in person.article_set.all  %}
    <div>{{ article.title }}</div>
  {% endfor %}

  <hr>
  
  <h2>{{ person.username }}'s 댓글</h2>
  {% for comment in person.comment_set.all  %}
    <div>{{ comment.content }}</div>
  {% endfor %}

  <hr>

  <h2>{{ person.username }}'s 좋아요한 게시글</h2>
  {% for article in person.like_articles.all %}
    <div>{{ article.title }}</div>
  {% endfor %}

  <hr>

  <a href="{% url 'articles:index' %}">BACK</a>
{% endblock content %}
```

<hr>

```html
<!-- accounts/templates/accounts/profile.html -->
<!-- 업그레이드 버전 -->

{% extends 'base.html' %}

{% block content %}
  <h1>{{ person.username }} 님의 프로필</h1>

  <hr>

  <h2>{{ person.username }}'s 게시글</h2>
  {% for article in person.article_set.all  %}
    {% comment %} <div>{{ article.title }}</div> {% endcomment %}
    {% comment %} <a href="{% url 'articles:detail' article.pk %}">{{ article.title }}</a>   {% endcomment %}
    <div class='card'>
      <div class='card-body'>
        <h5 class='card-title'>{{ article.title }}</h5>
        <p class = 'card-text'>{{ article.content }}</p>
        <a href="{% url 'articles:detail' article.pk %}" class="btn btn-primary">자세히 보기</a>
      </div>
    </div>
  {% endfor %}

  <hr>
  
  <h2>{{ person.username }}'s 댓글</h2>
  {% for comment in person.comment_set.all  %}
    <div>{{ comment.content }}</div>
  {% endfor %}

  <hr>

  <h2>{{ person.username }}'s 좋아요한 게시글</h2>
  {% for article in person.like_articles.all %}
    {% comment %} <div>{{ article.title }}</div> {% endcomment %}
    <div class='card'>
      <div class='card-body'>
        <h5 class='card-title'>{{ article.title }}</h5>
        <p class = 'card-text'>{{ article.content }}</p>
        <a href="{% url 'articles:detail' article.pk %}" class="btn btn-primary">자세히 보기</a>
      </div>
    </div>
  {% endfor %}
  
  <hr>
  
  <a href="{% url 'articles:index' %}" class='btn btn-success'>BACK</a>
{% endblock content %}
```

profile 템플릿으로 이동할 수 있는 하이퍼링크 작성  

```html
<!-- templates/base.html -->

...
    <h3 id="user-hello">
        <i>안녕하세요, <a href="{% url 'accounts:profile' user.username %}">{{user}}</a> 님 !</i>
    </h3>
...
```

```html
<!-- articles/index.html -->

...
    <p>
      [{{article.id}}] 
      <a href="{% url 'articles:detail' article.pk %}" id="article-title">{{article.title}}</a>
      - 작성자: <a href="{% url 'accounts:profile' article.user.username %}">{{article.user}}</a>
    </p>
...
```